In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl          
import matplotlib.pyplot as plt 
import seaborn as sns
import operator
import glob
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
%matplotlib inline 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.font_manager as fm

fontprop = fm.FontProperties(fname="IropkeBatangM.ttf", size=13)


필요한 라이브러리를 import한다.

In [2]:
##기상데이터 모두 합치기

# read files
weather_allFiles1 = glob.glob('./환경기상데이터/노원구/*')
weather_allFiles2 = glob.glob('./환경기상데이터/종로구/*')
allFiles = weather_allFiles1 + weather_allFiles2
w = []

for f in allFiles:
    temp = pd.read_csv(f)
    w.append(temp)

weather = pd.concat(w)

weather.drop(['flag', 'co2', 'vocs', 'noise', 'Unnamed: 10', 'temp','humi'], axis = 1, inplace=True)

# 이상치 제거
weather = weather[weather['pm10'] != -999]
weather['tm'] = weather['tm'] // 100
weather.head()

# 계층적 인덱스를 사용하지 않아서 reset_index() 로 인덱스를 칼럼으로 변환
# groupby serial & tm and average
weather = weather.groupby(['tm', 'serial']).mean().reset_index()

# serial number 를 행정동코드로 바꾼다.
weather.replace('V10O1610252',11110615, inplace = True)
weather.replace('V10O1610546',11110615, inplace = True)
weather.replace('V10O1610540',11110615, inplace = True)
weather.replace('V10O1610542',11110615, inplace = True)
weather.replace('V10O1610543',11110615, inplace = True)
weather.replace('V10O1610544',11110615, inplace = True)
weather.replace('V10O1610545',11110615, inplace = True)
weather.replace('V10O1610567',11110615, inplace = True)
weather.replace('V01o1610468',11110530, inplace = True)
weather.replace('V10O1611289',999, inplace = True)
weather.replace('V10O1611172',11110530, inplace = True)
weather.replace('V10O1611634',11110600, inplace = True)
weather.replace('V10O1611887',999, inplace = True)
weather.replace('V10O1611639',11110630, inplace = True)
weather.replace('V10O1611658',11110640, inplace = True)
weather.replace('V10O1612113',11110615, inplace = True)
weather.replace('V10O1611151',11110670, inplace = True)
weather.replace('V10O1611145',11110615, inplace = True)
weather.replace('V10O1611623',11110650, inplace = True)
weather.replace('V10O1611750',11110615, inplace = True)
weather.replace('V10O1611170',11110550, inplace = True)
weather.replace('V10O1611684',11110615, inplace = True)
weather.replace('V10O1611220',11110670, inplace = True)
weather.replace('V10O1612106',11110710, inplace = True)
weather.replace('V10O1611251',11110690, inplace = True)
weather.replace('V10O1611173',11110615, inplace = True)
weather.replace('V10O1611258',11110560, inplace = True)
weather.replace('V10O1611255',11110515, inplace = True)
weather.replace('V10O1611698',11110600, inplace = True)
weather.replace('V10O1611722',11110600, inplace = True)
weather.replace('V10O1611645',11110650, inplace = True)
weather.replace('V10O1610610',11350611, inplace = True)
weather.replace('V10O1610376',11350695, inplace = True)
weather.replace('V10O1610293',11350720, inplace = True)
weather.replace('V10O1610356',11350630, inplace = True)
weather.replace('V10O1610616',11350640, inplace = True)
weather.replace('V10O1610200',11350640, inplace = True)
weather.replace('V10O1610643',11350580, inplace = True)
weather.replace('V10O1610642',11350600, inplace = True)
weather.replace('V10O1610297',11350670, inplace = True)
weather.replace('V10O1610312',11350640, inplace = True)
weather.replace('V10O1610102',11350611, inplace = True)
weather.replace('V10O1610351',11350695, inplace = True)
weather.replace('V10O1610629',11350595, inplace = True)
weather.replace('V10O1610630',11350595, inplace = True)
weather.replace('V10O1611104',11350695, inplace = True)
weather.replace('V10O1611097',11350625, inplace = True)
weather.replace('V10O1611100',11350640, inplace = True)
weather.replace('V10O1612126',11350619, inplace = True)
weather.replace('V10O1611102',11350670, inplace = True)
weather.replace('V10O1611652',11350600, inplace = True)
weather.replace('V10O1611150',11350665, inplace = True)
weather.replace('V10O1611229',11350560, inplace = True)
location = list(weather['serial'].unique())
location.remove(999)
location_weather={}
for i in location:
    location_weather[i] = weather[weather['serial']==i].groupby('tm')['pm10','pm25'].mean().reset_index()

1. 모든 기상데이터를 합치고 필요한 컬럼만 남긴다.
2. 결측치(-999)의 인덱스는 제거해준다.
3. 다른 데이터 파일의 컬럼과 형식을 맞춰주기위해 컬럼명을 수정한다.
4. location_weather라는 딕셔너리를 만들고 지역별로 데이터프레임을 구분한다. 

In [3]:
##주말 날짜 구분    # 휴일 total_holiday, 주말 weekend_date , 공휴일 holiday, 평일 weekday
weather_copy = weather['tm']//100
weekdata_index = list(set(weather['tm']//100))
weekdata_index.sort()
weekend_data_saturday = []
weekend_data_sunday = []

for i in range(53):  #일요일
    weekend_data_sunday.append(weekdata_index[0+(7*i)])
for i in range(52): #토요일
    weekend_data_saturday.append(weekdata_index[6+(7*i)])
weekend_date = [] #토요일+일요일
for i in weekend_data_saturday:
    weekend_date.append(i)
for i in weekend_data_sunday:
    weekend_date.append(i)
weekend_date.sort()

holiday = [20180505, 20180507, 20180522, 20180606, 20180613, 20180815,
          20180923, 20180924, 20180925, 20180926, 20181003, 20181009, 20181225,
          20190101, 20190204, 20190205, 20190206, 20190301]

weekend_date_copy = weekend_date
for i in holiday:
    weekend_date_copy.append(i)
total_holiday = list(set(weekend_date_copy))
total_holiday = np.array(total_holiday)
total_holiday.sort()

weekday=weekdata_index
for i in total_holiday:
    weekday.remove(i)

In [4]:
d = glob.glob('./유동인구데이터/성연령유동/*')
a = glob.glob('./유동인구데이터/시간대유동/*')

w = []
g = []

for f in d:
    temp = pd.read_csv(f, sep='|')
    w.append(temp)
    
for f in a:
    temp = pd.read_csv(f, sep='|')
    g.append(temp)
    
age_data = pd.concat(w)
time_data = pd.concat(g)
age_data['HDONG_CD']= age_data['HDONG_CD']//100
time_data['HDONG_CD']= time_data['HDONG_CD']//100
age_data.drop('STD_YM', axis=1,inplace=True)
time_data.drop('STD_YM', axis=1,inplace=True)

1. 유동인구 데이터 파일들을 성별/시간대별로 합친다.
2. 지역코드를 통일한다.

In [ ]:
data= pd.read_csv('./CARD_SPENDING.txt', sep='\t')
data['GU_CD'] = data['GU_CD'] + 11000
data['GU_CD'] = data['GU_CD'] * 1000
data['GU_CD']= data['GU_CD'] + data['DONG_CD']
data = data.drop('DONG_CD', axis = 1)

카드데이터를 불러오고 지역정보를 통일한다.

In [ ]:
time_data['STD_YMD']=time_data['STD_YMD']*100
exist_weather_location=[]
for i in list(time_data['HDONG_CD'].unique()) :
    if i in list(location_weather.keys()):
        exist_weather_location.append(i)
total_exist_location={}
for i in  exist_weather_location:
    total_exist_location[i]=location_weather[i]

기상데이터의 지역정보와 유동인구데이터의 지역정보가 둘 다있는 경우
미세먼지관한 정보를 딕셔너리로 저장

In [ ]:
def hoilday_location(location):   
    holiday_location=[]
    for i in range(len(location['STD_YMD'])):
        if location.iloc[i]['STD_YMD']//100 in total_holiday:
            holiday_location.append(1)
        else:
            holiday_location.append(0)
    location['holiday']=holiday_location
    return location

해당 행정동 데이터에 휴일 정보 추가

In [ ]:
def location_amount(location_num):     
    total_exist_time=[]
    test = time_data[time_data['HDONG_CD']==location_num]
    for i in range(len(test)):
        for j in range(24):
            if j<10:
                total_exist_time.append([test.iloc[i]['STD_YMD']+j,test.iloc[i]['TMST_0'+str(j)]])
            else :
                total_exist_time.append([test.iloc[i]['STD_YMD']+j,test.iloc[i]['TMST_'+str(j)]])
    total_exist_time=np.array(total_exist_time)

    final=[]
    for i in range(len(total_exist_time[:,0])):
        if int(total_exist_time[i,0]) in list(total_exist_location[location_num]['tm']):
            d = total_exist_location[location_num][total_exist_location[location_num]['tm']==total_exist_time[i,0]]
            final.append([int(total_exist_time[i,0]), d['pm10'], d['pm25'],total_exist_time[i,1]])
    k = pd.DataFrame(final, columns=['STD_YMD','pm10','pm25','amount'])
    k['pm10'] =np.float64(k['pm10'])
    k['pm25'] =np.float64(k['pm25'])
    hoilday_location(k)
    return k

유동인구의 행정동과 환경 데이터의 행정동 정보가 둘 다 있는 경우 데이터 합치기

In [ ]:
for i in list(total_exist_location.keys()):            
    location_amount(i).to_csv('./'+str(i)+'데이터.csv')

행정동별 파일 생성

In [ ]:
time_data['STD_YMD']=time_data['STD_YMD']//100

amount_1111 = time_data[time_data['HDONG_CD']//10000 == 1111]
amount_1135 = time_data[time_data['HDONG_CD']//10000 == 1135]

amount_1111_detail = list(amount_1111['HDONG_NM'].unique())
amount_1135_detail = list(amount_1135['HDONG_NM'].unique())

종로구와 노원구를 나눔

In [ ]:
amount_1111_detail_holiday={}          ## 지역별 주말/평일 시간대별 유동인구 
amount_1111_detail_weekday={}
amount_1135_detail_holiday={}
amount_1135_detail_weekday={}

for k in amount_1111_detail:
    holi_case=[]
    week_case=[]
    for i in range(365):
        if amount_1111[amount_1111['HDONG_NM']==k]['STD_YMD'].iloc[i] in total_holiday:
            holi_case.append(amount_1111[amount_1111['HDONG_NM']==k].iloc[i])
        else:
            week_case.append(amount_1111[amount_1111['HDONG_NM']==k].iloc[i])
    amount_1111_detail_holiday[k]=pd.DataFrame(holi_case)
    amount_1111_detail_weekday[k]=pd.DataFrame(week_case)
    
for k in amount_1135_detail:
    holi_case=[]
    week_case=[]
    for i in range(365):
        if amount_1135[amount_1135['HDONG_NM']==k]['STD_YMD'].iloc[i] in total_holiday:
            holi_case.append(amount_1135[amount_1135['HDONG_NM']==k].iloc[i])
        else:
            week_case.append(amount_1135[amount_1135['HDONG_NM']==k].iloc[i])
    amount_1135_detail_holiday[k]=pd.DataFrame(holi_case)
    amount_1135_detail_weekday[k]=pd.DataFrame(week_case)

지역별 주말/평일 시간대별 유동인구 

In [ ]:
month= [20180401,20180501,20180601,20180701,20180801,20180901,20181001,20181101,20181201,20190101,20190201,20190301,20190401]

In [ ]:
def monthly_info(location) :  
    monthly_location = {}
    location = location.drop('HDONG_CD', axis=1)
    month_order = [4,5,6,7,8,9,10,11,12,1,2,3]
    for i in range(len(month)-1):
        amount = location[(location['STD_YMD']<month[i+1]) & (location['STD_YMD']>=month[i])].mean()
        amount.drop('STD_YMD', inplace=True)
        monthly_location[month_order[i]]=amount
    return monthly_location

월별 유동인구 정보

In [ ]:
for j in list(amount_1111_detail_holiday.keys()):
    info_holi = monthly_info(amount_1111_detail_holiday[j])
    info_week = monthly_info(amount_1111_detail_weekday[j])
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    for i in range(12):
        info_holi[i+1].plot()
    plt.legend([1,2,3,4,5,6,7,8,9,10,11,12])
    plt.title(j+'_주말 시간대별 유동인구(월별)' , fontproperties=fontprop)
    plt.subplot(1,2,2)
    for i in range(12):
        info_week[i+1].plot(title = 'weekday')
    plt.legend([1,2,3,4,5,6,7,8,9,10,11,12])
    plt.title(j+'_평일 시간대별 유동인구(월별)' , fontproperties=fontprop)

월별 주말/평일 유동인구  종로구

In [ ]:
for j in list(amount_1135_detail_holiday.keys()):
    info_holi = monthly_info(amount_1135_detail_holiday[j])
    info_week = monthly_info(amount_1135_detail_weekday[j])
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    for i in range(12):
        info_holi[i+1].plot()
    plt.legend([1,2,3,4,5,6,7,8,9,10,11,12])
    plt.title(j+'_주말 시간대별 유동인구(월별)' , fontproperties=fontprop)
    plt.subplot(1,2,2)
    for i in range(12):
        info_week[i+1].plot(title = 'weekday')
    plt.legend([1,2,3,4,5,6,7,8,9,10,11,12])
    plt.title(j+'_평일 시간대별 유동인구(월별)' , fontproperties=fontprop)

월별 주말/평일 유동인구  노원구

In [ ]:
card_종로 = data[data['GU_CD']//10000 == 1111]
card_노원 = data[data['GU_CD']//10000 == 1135]
card_종로_품목별 = {}
card_노원_품목별 = {}

for i in list(card_종로['MCT_CAT_CD'].unique()):
    card_종로_품목별[i] = card_종로[card_종로['MCT_CAT_CD']==i].groupby('STD_DD')['USE_CNT','USE_AMT'].sum()

for i in list(card_노원['MCT_CAT_CD'].unique()):
    card_노원_품목별[i] = card_노원[card_노원['MCT_CAT_CD']==i].groupby('STD_DD')['USE_CNT','USE_AMT'].sum()

종로/노원 지역별 매출, 결제횟수 평균을 딕셔너리로 저장

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
for i in list(card_종로_품목별.keys()):
    plt.bar(str(i),card_종로_품목별[i]['USE_AMT'].sum())
plt.title('종로구 품목별 매출' , fontproperties=fontprop)
plt.ylabel('amount')
plt.xlabel('code')

plt.subplot(1,2,2)
for i in list(card_종로_품목별.keys()):
    plt.bar(str(i),card_종로_품목별[i]['USE_CNT'].sum())
plt.title('종로구 품목별 결제횟수' , fontproperties=fontprop)
plt.ylabel('count')
plt.xlabel('code')

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
for i in list(card_노원_품목별.keys()):
    plt.bar(str(i),card_노원_품목별[i]['USE_AMT'].sum())
plt.title('노원구 품목별 매출' , fontproperties=fontprop)
plt.ylabel('amount')
plt.xlabel('code')

plt.subplot(1,2,2)
for i in list(card_노원_품목별.keys()):
    plt.bar(str(i),card_노원_품목별[i]['USE_CNT'].sum())
plt.title('노원구 품목별 결제횟수' , fontproperties=fontprop)
plt.ylabel('count')
plt.xlabel('code')


In [ ]:
weather_종로 = weather[weather['serial']//10000 == 1111]
weather_노원 = weather[weather['serial']//10000 == 1135]
weather_종로['tm']=  weather_종로['tm']//100
weather_노원['tm']=  weather_노원['tm']//100

지역별 날씨 구분

In [ ]:
def monthly_pm_info(location) :  #월별 미세먼지 평균
    monthly_location = []
    monthly_location = []
    month_order = [4,5,6,7,8,9,10,11,12,1,2,3]
    for i in range(len(month)-1):
        amount = location[(location['tm']<month[i+1]) & (location['tm']>=month[i])].mean()
        amount = amount.drop('tm')
        monthly_location.append([str(month_order[i]) , amount])
    return monthly_location

  #월별 미세먼지 평균

In [ ]:
pm_종로_월별_평균 =  monthly_pm_info(weather_종로.groupby('tm')['pm10','pm25'].mean().reset_index())
pm_노원_월별_평균 =  monthly_pm_info(weather_노원.groupby('tm')['pm10','pm25'].mean().reset_index())

In [ ]:

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
month_info=[]
month_pm10_종로=[]
month_pm25_종로=[]
month_pm10_노원=[]
month_pm25_노원=[]
for i in pm_종로_월별_평균:
    month_info.append(i[0])
    month_pm10_종로.append(i[1]['pm10'])
    month_pm25_종로.append(i[1]['pm25'])
for i in pm_노원_월별_평균:
    month_pm10_노원.append(i[1]['pm10'])
    month_pm25_노원.append(i[1]['pm25'])
plt.plot(month_info, month_pm10_종로, color='tomato')
plt.plot(month_info, month_pm10_노원, color='steelblue')
plt.title('pm10')
plt.legend(['Jong','Nowon'])
plt.xlabel('month')
plt.subplot(1,2,2)
plt.title('pm25')
plt.plot(month_info, month_pm25_종로, color='tomato')
plt.plot(month_info, month_pm25_노원, color='steelblue')
plt.legend(['Jong','Nowon'])
plt.xlabel('month')


월별 미세먼지 평균 농도

In [ ]:
def pm_amount_1111_graph(code):
    name = time_data[time_data['HDONG_CD']==code]['HDONG_NM'].iloc[0]
    amount_행정동_휴일 = amount_1111_detail_holiday[name].drop(['HDONG_CD','HDONG_NM'], axis=1)
    amount_행정동_평일 = amount_1111_detail_weekday[name].drop(['HDONG_CD','HDONG_NM'], axis=1)
    pm_행정동 = location_weather[time_data[time_data['HDONG_NM']==name]['HDONG_CD'].unique()[0]]
    pm_행정동['tm'] = pm_행정동['tm']//100
    pm_행정동  = pm_행정동.groupby('tm')['pm10','pm25'].mean().reset_index()

    pm_high_date=[]
    pm_middle_date=[]
    pm_low_date=[]
    for i in list(pm_행정동.index):
        if pm_행정동.loc[i]['pm10'] > 80 or pm_행정동.loc[i]['pm25'] > 35:
             pm_high_date.append(int(pm_행정동.loc[i]['tm']))
        elif pm_행정동.loc[i]['pm10'] < 30 and pm_행정동.loc[i]['pm25'] < 15:
             pm_low_date.append(int(pm_행정동.loc[i]['tm']))
        else: 
             pm_middle_date.append(int(pm_행정동.loc[i]['tm']))
                                   
    pm_high_holiday = []
    pm_middle_holiday = []
    pm_low_holiday = []
    
    for i in total_holiday:
        if i in list(pm_high_date):
            pm_high_holiday.append(i)
        elif i in list(pm_middle_date):    
            pm_middle_holiday.append(i)
        elif i in list(pm_low_date):
            pm_low_holiday.append(i)

    pm_high_weekday = []
    pm_middle_weekday = []
    pm_low_weekday = []
    
    for i in weekday:
        if i in list(pm_high_date):
            pm_high_weekday.append(i)
        elif i in list(pm_middle_date):    
            pm_middle_weekday.append(i)
        elif i in list(pm_low_date):
            pm_low_weekday.append(i)

    pm_high_amount_weekday= []
    pm_middle_amount_weekday= []
    pm_low_amount_weekday= []
    pm_high_amount_holiday= []
    pm_middle_amount_holiday= []
    pm_low_amount_holiday= []

    for i in pm_high_weekday:
        pm_high_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])
    for i in pm_middle_weekday:
        pm_middle_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])
    for i in pm_low_weekday:
        pm_low_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])

    for i in pm_high_holiday:
        pm_high_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])
    for i in pm_middle_holiday:
        pm_middle_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])
    for i in pm_low_holiday:
        pm_low_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])

    pm_high_amount_weekday= pd.concat(pm_high_amount_weekday)
    pm_middle_amount_weekday= pd.concat(pm_middle_amount_weekday)
    pm_low_amount_weekday= pd.concat(pm_low_amount_weekday)
    pm_high_amount_holiday= pd.concat(pm_high_amount_holiday)
    pm_middle_amount_holiday=pd.concat(pm_middle_amount_holiday)
    pm_low_amount_holiday= pd.concat(pm_low_amount_holiday)
    
    pm_high_amount_holiday = pm_high_amount_holiday.drop('STD_YMD',axis=1)
    pm_middle_amount_holiday = pm_middle_amount_holiday.drop('STD_YMD',axis=1)
    pm_low_amount_holiday = pm_low_amount_holiday.drop('STD_YMD',axis=1)
    pm_high_amount_weekday = pm_high_amount_weekday.drop('STD_YMD',axis=1)
    pm_middle_amount_weekday = pm_middle_amount_weekday.drop('STD_YMD',axis=1)
    pm_low_amount_weekday = pm_low_amount_weekday.drop('STD_YMD',axis=1)
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    pm_high_amount_holiday.mean().plot()
    pm_middle_amount_holiday.mean().plot()
    pm_low_amount_holiday.mean().plot()
    plt.title(name+' 휴일 시간대별 유동인구' , fontproperties=fontprop)
    plt.legend(['high', 'middle','low'])
    plt.subplot(1,2,2)
    pm_high_amount_weekday.mean().plot()
    pm_middle_amount_weekday.mean().plot()
    pm_low_amount_weekday.mean().plot()
    plt.title(name+' 평일 시간대별 유동인구' , fontproperties=fontprop)
    plt.legend(['high', 'middle','low'])

종로구 휴일, 평일 시간대별 유동인구 시각화 함수

In [ ]:
def pm_amount_1135_graph(code):
    name = time_data[time_data['HDONG_CD']==code]['HDONG_NM'].iloc[0]
    amount_행정동_휴일 = amount_1135_detail_holiday[name].drop(['HDONG_CD','HDONG_NM'], axis=1)
    amount_행정동_평일 = amount_1135_detail_weekday[name].drop(['HDONG_CD','HDONG_NM'], axis=1)
    pm_행정동 = location_weather[time_data[time_data['HDONG_NM']==name]['HDONG_CD'].unique()[0]]
    pm_행정동['tm'] = pm_행정동['tm']//100
    pm_행정동  = pm_행정동.groupby('tm')['pm10','pm25'].mean().reset_index()

    pm_high_date=[]
    pm_middle_date=[]
    pm_low_date=[]
    for i in list(pm_행정동.index):
        if pm_행정동.loc[i]['pm10'] > 80 or pm_행정동.loc[i]['pm25'] > 35:
             pm_high_date.append(int(pm_행정동.loc[i]['tm']))
        elif pm_행정동.loc[i]['pm10'] < 30 and pm_행정동.loc[i]['pm25'] < 15:
             pm_low_date.append(int(pm_행정동.loc[i]['tm']))
        else: 
             pm_middle_date.append(int(pm_행정동.loc[i]['tm']))
                                   
    pm_high_holiday = []
    pm_middle_holiday = []
    pm_low_holiday = []
    
    for i in total_holiday:
        if i in list(pm_high_date):
            pm_high_holiday.append(i)
        elif i in list(pm_middle_date):    
            pm_middle_holiday.append(i)
        elif i in list(pm_low_date):
            pm_low_holiday.append(i)

    pm_high_weekday = []
    pm_middle_weekday = []
    pm_low_weekday = []
    
    for i in weekday:
        if i in list(pm_high_date):
            pm_high_weekday.append(i)
        elif i in list(pm_middle_date):    
            pm_middle_weekday.append(i)
        elif i in list(pm_low_date):
            pm_low_weekday.append(i)

    pm_high_amount_weekday= []
    pm_middle_amount_weekday= []
    pm_low_amount_weekday= []
    pm_high_amount_holiday= []
    pm_middle_amount_holiday= []
    pm_low_amount_holiday= []

    for i in pm_high_weekday:
        pm_high_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])
    for i in pm_middle_weekday:
        pm_middle_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])
    for i in pm_low_weekday:
        pm_low_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])

    for i in pm_high_holiday:
        pm_high_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])
    for i in pm_middle_holiday:
        pm_middle_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])
    for i in pm_low_holiday:
        pm_low_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])

    pm_high_amount_weekday= pd.concat(pm_high_amount_weekday)
    pm_middle_amount_weekday= pd.concat(pm_middle_amount_weekday)
    pm_low_amount_weekday= pd.concat(pm_low_amount_weekday)
    pm_high_amount_holiday= pd.concat(pm_high_amount_holiday)
    pm_middle_amount_holiday=pd.concat(pm_middle_amount_holiday)
    pm_low_amount_holiday= pd.concat(pm_low_amount_holiday)
    
    pm_high_amount_holiday = pm_high_amount_holiday.drop('STD_YMD',axis=1)
    pm_middle_amount_holiday = pm_middle_amount_holiday.drop('STD_YMD',axis=1)
    pm_low_amount_holiday = pm_low_amount_holiday.drop('STD_YMD',axis=1)
    pm_high_amount_weekday = pm_high_amount_weekday.drop('STD_YMD',axis=1)
    pm_middle_amount_weekday = pm_middle_amount_weekday.drop('STD_YMD',axis=1)
    pm_low_amount_weekday = pm_low_amount_weekday.drop('STD_YMD',axis=1)
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    pm_high_amount_holiday.mean().plot(color='steelblue')
    pm_middle_amount_holiday.mean().plot(color='darkorange')
    pm_low_amount_holiday.mean().plot(color='g')
    plt.title(name+' 휴일 시간대별 유동인구' , fontproperties=fontprop)
    plt.legend(['high', 'middle','low'])
    plt.subplot(1,2,2)
    pm_high_amount_weekday.mean().plot(color='steelblue')
    pm_middle_amount_weekday.mean().plot(color='darkorange')
    pm_low_amount_weekday.mean().plot(color='g')
    plt.title(name+' 평일 시간대별 유동인구' , fontproperties=fontprop)
    plt.legend(['high', 'middle','low'])

노원구 휴일, 평일 시간대별 유동인구 시각화

In [ ]:
exist_1111=[]
exist_1135=[]

for i in list(location_weather.keys()):
    if i <11350000:
        exist_1111.append(i)
    else:
        exist_1135.append(i)

In [ ]:
for i in exist_1111:
    pm_amount_1111_graph(i)

시각화

In [ ]:
for k in exist_1135:
    pm_행정동 = location_weather[k]
    pm_행정동['tm'] = pm_행정동['tm']//100
    pm_행정동  = pm_행정동.groupby('tm')['pm10','pm25'].mean().reset_index()

    pm_high_date=[]
    pm_middle_date=[]
    pm_low_date=[]
    for i in list(pm_행정동.index):
        if pm_행정동.loc[i]['pm10'] > 80 or pm_행정동.loc[i]['pm25'] > 35:
            pm_high_date.append(int(pm_행정동.loc[i]['tm']))
        elif pm_행정동.loc[i]['pm10'] < 30 and pm_행정동.loc[i]['pm25'] < 15:
            pm_low_date.append(int(pm_행정동.loc[i]['tm']))
        else: 
            pm_middle_date.append(int(pm_행정동.loc[i]['tm']))
                                   
    pm_high_holiday = []
    pm_middle_holiday = []
    pm_low_holiday = []

    name = time_data[time_data['HDONG_CD']==k]['HDONG_NM'].iloc[0]
    amount_행정동_휴일 = amount_1135_detail_holiday[name].drop(['HDONG_CD','HDONG_NM'], axis=1)
    amount_행정동_평일 = amount_1135_detail_weekday[name].drop(['HDONG_CD','HDONG_NM'], axis=1)
  
    for i in total_holiday:
        if i in list(pm_high_date):
            pm_high_holiday.append(i)
        elif i in list(pm_middle_date):    
            pm_middle_holiday.append(i)
        elif i in list(pm_low_date):
            pm_low_holiday.append(i)

    pm_high_weekday = []
    pm_middle_weekday = []
    pm_low_weekday = []
    
    for i in weekday:
        if i in list(pm_high_date):
            pm_high_weekday.append(i)
        elif i in list(pm_middle_date):    
            pm_middle_weekday.append(i)
        elif i in list(pm_low_date):
            pm_low_weekday.append(i)

    pm_high_amount_weekday= []
    pm_middle_amount_weekday= []
    pm_low_amount_weekday= []
    pm_high_amount_holiday= []
    pm_middle_amount_holiday= []
    pm_low_amount_holiday= []

    for i in pm_high_weekday:
        pm_high_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])
    for i in pm_middle_weekday:
        pm_middle_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])
    for i in pm_low_weekday:
        pm_low_amount_weekday.append(amount_행정동_평일[amount_행정동_평일['STD_YMD']==i])

    for i in pm_high_holiday:
        pm_high_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])
    for i in pm_middle_holiday:
        pm_middle_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])
    for i in pm_low_holiday:
        pm_low_amount_holiday.append(amount_행정동_휴일[amount_행정동_휴일['STD_YMD']==i])

    pm_high_amount_weekday= pd.concat(pm_high_amount_weekday)
    pm_middle_amount_weekday= pd.concat(pm_middle_amount_weekday)
    pm_low_amount_weekday= pd.concat(pm_low_amount_weekday)
    pm_high_amount_holiday= pd.concat(pm_high_amount_holiday)
    pm_middle_amount_holiday=pd.concat(pm_middle_amount_holiday)
    pm_low_amount_holiday= pd.concat(pm_low_amount_holiday)
    
    pm_high_amount_holiday = pm_high_amount_holiday.drop('STD_YMD',axis=1)
    pm_middle_amount_holiday = pm_middle_amount_holiday.drop('STD_YMD',axis=1)
    pm_low_amount_holiday = pm_low_amount_holiday.drop('STD_YMD',axis=1)
    pm_high_amount_weekday = pm_high_amount_weekday.drop('STD_YMD',axis=1)
    pm_middle_amount_weekday = pm_middle_amount_weekday.drop('STD_YMD',axis=1)
    pm_low_amount_weekday = pm_low_amount_weekday.drop('STD_YMD',axis=1)
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    pm_high_amount_holiday.mean().plot()
    pm_middle_amount_holiday.mean().plot()
    pm_low_amount_holiday.mean().plot()
    plt.title(name+' 휴일 시간대별 유동인구' , fontproperties=fontprop)
    plt.legend(['high', 'middle','low'])
    plt.subplot(1,2,2)
    pm_high_amount_weekday.mean().plot()
    pm_middle_amount_weekday.mean().plot()
    pm_low_amount_weekday.mean().plot()
    plt.title(name+' 평일 시간대별 유동인구' , fontproperties=fontprop)
    plt.legend(['high', 'middle','low'])

시각화

In [ ]:
amount_가회동 = data[data['GU_CD']==11110600].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_부암동 = data[data['GU_CD']==11110550].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_사직동 = data[data['GU_CD']==11110530].groupby('MCT_CAT_CD')['USE_CNT'].sum()

가회동, 부암동, 사직동 결제횟수 통계

In [ ]:
amount_숭인2동 = data[data['GU_CD']==11110710].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_이화동 = data[data['GU_CD']==11110640].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_종로5_6가동 = data[data['GU_CD']==11110630].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_창신1동 = data[data['GU_CD']==11110670].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_창신3동 = data[data['GU_CD']==11110690].groupby('MCT_CAT_CD')['USE_CNT'].sum()

숭인2동, 이화동, 종로5,6가동, 창신1동, 창신3동 결제횟수 통계

In [ ]:
plt.figure(figsize=(13,18))
plt.subplot(3,2,1)
amount_숭인2동.sort_values(ascending=False).plot.bar()
plt.title('숭인2동 결제횟수' , fontproperties=fontprop)
plt.subplot(3,2,2)
amount_이화동.sort_values(ascending=False).plot.bar()
plt.title('이화동 결제횟수' , fontproperties=fontprop)

plt.subplot(3,2,3)
amount_종로5_6가동.sort_values(ascending=False).plot.bar()
plt.title('종로5,6가동 결제횟수' , fontproperties=fontprop)

plt.subplot(3,2,4)
amount_창신1동.sort_values(ascending=False).plot.bar()
plt.title('창신1동 결제횟수' , fontproperties=fontprop)

plt.subplot(3,2,5)
amount_창신3동.sort_values(ascending=False).plot.bar()
plt.title('창신3동 결제횟수' , fontproperties=fontprop)


In [ ]:
plt.figure(figsize=(13,12))
plt.subplot(2,2,1)
amount_가회동.sort_values(ascending=False).plot.bar()
plt.title('가회동 결제횟수' , fontproperties=fontprop)
plt.subplot(2,2,2)
amount_부암동.sort_values(ascending=False).plot.bar()
plt.title('부암동 결제횟수' , fontproperties=fontprop)
plt.subplot(2,2,3)
amount_사직동.sort_values(ascending=False).plot.bar()
plt.title('사직동 결제횟수' , fontproperties=fontprop)

시각화

In [ ]:
amount_중계본동 = data[data['GU_CD']==11350619].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_중계본동.sort_values(ascending=False).plot.bar()
plt.title('중계본동 결제횟수' , fontproperties=fontprop)

amount_공릉1동 = data[data['GU_CD']==11350595].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_상계3_4동 = data[data['GU_CD']==11350665].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_상계5동 = data[data['GU_CD']==11350670].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_상계10동 = data[data['GU_CD']==11350720].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_월계1동 = data[data['GU_CD']==11350560].groupby('MCT_CAT_CD')['USE_CNT'].sum()
amount_하계1동 = data[data['GU_CD']==11350611].groupby('MCT_CAT_CD')['USE_CNT'].sum()


노원구 결제횟수 통계

In [ ]:
plt.figure(figsize=(13,18))
plt.subplot(3,2,1)
amount_공릉1동.sort_values(ascending=False).plot.bar()
plt.title('공릉1동 결제횟수' , fontproperties=fontprop)
plt.subplot(3,2,2)
amount_상계3_4동.sort_values(ascending=False).plot.bar()
plt.title('상계3,4동 결제횟수' , fontproperties=fontprop)

plt.subplot(3,2,3)
amount_상계5동.sort_values(ascending=False).plot.bar()
plt.title('상계5동 결제횟수' , fontproperties=fontprop)

plt.subplot(3,2,4)
amount_상계10동.sort_values(ascending=False).plot.bar()
plt.title('상계10동 결제횟수' , fontproperties=fontprop)

plt.subplot(3,2,5)
amount_월계1동.sort_values(ascending=False).plot.bar()
plt.title('월계1동 결제횟수' , fontproperties=fontprop)

plt.subplot(3,2,6)
amount_하계1동.sort_values(ascending=False).plot.bar()
plt.title('하계1동 결제횟수' , fontproperties=fontprop)


시각화

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
data[data['GU_CD']==11110550].groupby('MCT_CAT_CD')['USE_AMT'].sum().sort_values(ascending=False).plot.bar()
plt.title('부암동 결제총액' , fontproperties=fontprop)
plt.subplot(1,2,2)
data[data['GU_CD']==11110550].groupby('MCT_CAT_CD')['USE_CNT'].sum().sort_values(ascending=False).plot.bar()
plt.title('부암동 결제횟수' , fontproperties=fontprop)

부암동 매출 정보

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
data[data['GU_CD']//10000==1135].groupby('MCT_CAT_CD')['USE_AMT'].sum().sort_values(ascending=False).plot.bar()
plt.title('노원구 결제총액' , fontproperties=fontprop)
plt.subplot(1,2,2)
data[data['GU_CD']//10000==1135].groupby('MCT_CAT_CD')['USE_CNT'].sum().sort_values(ascending=False).plot.bar()
plt.title('노원구 결제횟수' , fontproperties=fontprop)

노원구 매출 분포